In [1]:
from files.utils.constants import *
from files.utils.utility_functions import *
from files.utils.dataset_3d_helper import *
from files.pif.pif import *

# NOTE: Using more than 195 models, Voronoi tesselation does not work

In [2]:
ds, gt = create_plane(num_points=1000, of=0.5)
plot3d(ds, gt)

In [3]:
from files.classes.base_models import *
PlaneEstimator()

In [4]:
from sklearn.ensemble import IsolationForest as iFor


params = {
    "mss": 3,
    "num_models": 195,
    "training_epochs": 0,
}

ifor = iFor().fit(ds)
scores = -ifor.score_samples(ds)
ifor_auc, *_ = make_roc(gt, scores, show=False, to_plot=False);
print("iFor AUC: ", ifor_auc)

iFor AUC:  0.722124


In [5]:
pif = PreferenceIsolationForest(data=ds, model_name=PLANE)
scores = pif.anomaly_detection(params=params, in_th=0.1)

Anomaly Detection
Hello
--------------------------------------------------
Building RanSac models
Iteration 195/195: [==============================] (100%) ETA: 0:00:00.202929
--------------------------------------------------
Building preference matrix
--------------------------------------------------
Building Voronoi Forest
--------------------------------------------------
Done


In [6]:
pif_auc, *_ = make_roc(gt, scores, show=False, to_plot=False);
print("PIF AUC: ", pif_auc)

PIF AUC:  0.9303319999999999
